In [1]:
import google.generativeai as genai
import pandas as pd
from copy import deepcopy


C:\Users\wangs\anaconda3\envs\gemini-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prompt preparation

In [2]:
def remove_non_ascii(sentence):
    return ''.join(char for char in sentence if ord(char) < 128)

In [5]:
# with open('prompt/prompt_system_etd.txt', 'r') as fp:
#     etd_system = fp.readlines()
    
# with open('prompt/prompt_system_ps.txt', 'r') as fp:
#     ps_system = fp.readlines()
    
with open('prompt/prompt_system_etd_noCoT.txt', 'r') as fp:
    etd_system = fp.readlines()
    
with open('prompt/prompt_system_ps_noCoT.txt', 'r') as fp:
    ps_system = fp.readlines()

In [ ]:
prompt_parts = []
prompt_parts.append(" ".join(etd_system))
prompt_parts

In [6]:
prompt_parts = []
prompt_parts.append(" ".join(ps_system))
prompt_parts

['Please determine whether the given user input provides the information of the programâ€™s malfunction or any problematic scenario currently encountered by the user. If provided, return "The answer is YES"; otherwise, return "The answer is No".']

In [7]:
df_issue = pd.read_csv("prompt/issue.csv", encoding = "ISO-8859-1")
# df_issue = pd.read_csv("prompt/issue_wrong.csv")

# df_etd_example = pd.read_csv("experiment/random_pattern/ETD/gemini/etd_prompt_random3_with_negno11_gemini.csv")
df_ps_example = pd.read_csv("experiment/random_pattern/PS/gemini/ps_prompt_random_noCoT_gemini.csv")

In [8]:
len(df_issue)

1000

In [9]:
issue_clean = [remove_non_ascii(issue) for issue in df_issue["issue"]]
df_issue["issue"] = issue_clean
df_issue.head()

,issue
0,"Hi, when I am trying to apply class dynamicall..."
1,is it possible to get a RouteConfig matched ag...
2,I have angular running from a .Net Core server...
3,"Hello everyone, I want to do sub menu with sea..."
4,I'm struggling with getting this logic [code]....


In [ ]:
df_etd_example = df_etd_example.sample(frac=1)
df_etd_example.reset_index(drop=True, inplace=True)
df_etd_example

In [10]:
df_ps_example = df_ps_example.sample(frac=1)
df_ps_example.reset_index(drop=True, inplace=True)
df_ps_example

,Pattern,issue,output,1_example,2_examples,5_examples,6_examples,12_examples,14_examples
0,PS_SEEMS_TO_NEG_VERB,""""""" Hey guys, I'm trying to get a reference to...",The answer is YES.,0,0,0,0,1,1
1,PS_NEG_VERB,""""""" Hey guys, any ideas on why a test executes...",The answer is YES.,0,0,1,1,1,1
2,PS_NEG_VERB,""""""" Are there performance gains to be had by b...",The answer is NO.,0,0,0,0,0,0
3,PS_ONLY,""""""" Hi guys, I'm getting ready to launch the f...",The answer is YES.,1,1,1,1,1,1
4,PS_NEG_ADV_ADJ,""""""" Anyone good with git? I'm getting some wei...",The answer is YES.,0,0,1,1,1,1
5,PS_ONLY,""""""" Does anyone know of a way to determine if ...",The answer is NO.,0,0,0,1,0,0
6,PS_PROBLEM_IS,""""""" Anyone knows a good solution to use php-cl...",The answer is YES.,0,0,1,1,0,1
7,NaN,""""""" Hello! If I'm looking to test a component ...",The answer is NO.,0,0,0,0,1,0
8,PS_NEG_AUX_VERB,""""""" The ethereum wallet isn't finishing downlo...",The answer is YES.,0,0,0,0,1,1
9,PS_NO_NOUN,""""""" Oooph. So I narrowed down my previous Fire...",The answer is YES.,0,0,1,1,1,1


### for all examples

In [ ]:
chat_issue = ["input: " + q_str for q_str in list(df_etd_example["issue"].values)]
chat_issue

In [ ]:
chat_output = ["output: " + o_str for o_str in list(df_etd_example["output"].values)]
chat_output

In [ ]:
# chat_issue = list(df_etd_example["issue"].values)
chat_issue = ["input: " + q_str for q_str in list(df_ps_example["issue"].values)]
chat_issue

In [ ]:
# chat_output = list(df_etd_example["output"].values)
chat_output = ["output: " + o_str for o_str in list(df_ps_example["output"].values)]
chat_output

### for random/frequent patterns

In [ ]:
df_etd_randexample = df_etd_example[df_etd_example["1_example"]==1]

chat_issue = ["input: " + q_str for q_str in list(df_etd_randexample["issue"].values)]
# chat_issue.extend(list(df_etd_randexample["issue_n"].values))
chat_issue

In [ ]:
chat_output = ["output: " + o_str for o_str in list(df_etd_randexample["output"].values)]
# chat_output.extend(list(df_etd_randexample["output_n"].values))
chat_output

In [12]:
df_ps_randexample = df_ps_example[df_ps_example["1_example"]==1]

chat_issue = ["input: " + q_str for q_str in list(df_ps_randexample["issue"].values)]
# chat_issue.extend(list(df_ps_randexample["issue_n"].values))
chat_issue

['input: """ Hi guys, I\'m getting ready to launch the first really big angular2 app into prodution, but looking at the javascript timeline, the evaluate script of the whole bundle (300kB gzipped) takes 800ms, compared to the (more or less identical) AngularJS implementation that only took 40ms to evaluate this is quite a lot, any ideas? """']

In [13]:
chat_output = ["output: " + o_str for o_str in list(df_ps_randexample["output"].values)]
# chat_output.extend(list(df_ps_randexample["output_n"].values))
chat_output

['output: The answer is YES.']

## Model running

In [30]:
with open("gemini_token.txt", 'r') as fp:
    gemini_token = fp.readline()

In [31]:
genai.configure(api_key=gemini_token)

In [16]:
# Set up the model
generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 50,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

# Create GenerativeModel object
model = genai.GenerativeModel(
    model_name="gemini-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

In [17]:
for eg_input, eg_output in zip(chat_issue, chat_output):
    prompt_parts.append(eg_input)
    prompt_parts.append(eg_output)

prompt_parts

['Please determine whether the given user input provides the information of the programâ€™s malfunction or any problematic scenario currently encountered by the user. If provided, return "The answer is YES"; otherwise, return "The answer is No".',
 'input: """ Hi guys, I\'m getting ready to launch the first really big angular2 app into prodution, but looking at the javascript timeline, the evaluate script of the whole bundle (300kB gzipped) takes 800ms, compared to the (more or less identical) AngularJS implementation that only took 40ms to evaluate this is quite a lot, any ideas? """',
 'output: The answer is YES.']

In [ ]:
ans1= []
start_i = 61
for i in range(start_i,len(df_issue),1):
# for i in range(0,10,1):
    issue = df_issue.loc[i]["issue"]
    input_str = "input: \"\"\" {} \"\"\"".format(issue)
    
    prompt = deepcopy(prompt_parts)
    prompt.append(input_str)
    prompt.append("output: ")
    
    response = model.generate_content(prompt)
    ans1.append(response.text)

In [32]:
import time
from google.api_core.exceptions import GoogleAPIError

ans1= []
start_i = 0
flag = True

while flag:
    try:
        for i in range(start_i,len(df_issue),1):
#         for i in range(start_i,start_i+7,1):
            issue = df_issue.loc[i]["issue"]
            input_str = "input: \"\"\" {} \"\"\"".format(issue)
    
            prompt = deepcopy(prompt_parts)
            prompt.append(input_str)
            prompt.append("output: ")
    
            response = model.generate_content(prompt)
            ans1.append(response.text)
        
        flag = False
    except GoogleAPIError as error:
        if error.code == 429:  # Check for 429 (Rate Limit Exceeded) error
            print("Rate limit exceeded:", error.message)
            time.sleep(60)
            start_i = len(ans1)+0      
    except Exception as e:
        print(f"An error occurred: {e}")
        break  # Exit the loop on other errors

Rate limit exceeded: Resource has been exhausted (e.g. check quota).
Rate limit exceeded: Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:179991646430'.
Rate limit exceeded: Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:179991646430'.
Rate limit exceeded: Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:179991646430'.
Rate limit exceeded: Resource has been exhausted (e.g. check quota).
Rate limit exceeded: Quota exceeded for quota metric 'Generate Content API requests per minute' and lim

Rate limit exceeded: Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:179991646430'.
Rate limit exceeded: Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:179991646430'.
Rate limit exceeded: Resource has been exhausted (e.g. check quota).
Rate limit exceeded: Resource has been exhausted (e.g. check quota).
Rate limit exceeded: Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:179991646430'.
Rate limit exceeded: Quota exceeded for quota metric 'Generate Content API requests per minute' and lim

In [33]:
print(len(ans1))

1000


In [34]:
df = pd.DataFrame()
df["answer1"] = ans1

df.head(10)

,answer1
0,The answer is YES.
1,The answer is No.
2,The answer is No.
3,The answer is YES.
4,The answer is No.
5,The answer is No.
6,The answer is YES.
7,The answer is YES.
8,The answer is No.
9,The answer is YES.


In [35]:
df.to_csv("experiment/random_pattern/answer_rand.csv",index = None)

In [5]:
from sklearn import metrics as mt

In [4]:
# ETD results

df_test_random = pd.read_csv("experiment/random_pattern/ETD/gemini/etd_random2_withnegno1211_gemini_result(t=0.8).csv")
predictions_etd = df_test_random["y''_ETD"]
y_test = df_test_random["y_ETD"]
precison_etd = mt.precision_score(y_test, predictions_etd)
recall_etd = mt.recall_score(y_test, predictions_etd)
score_etd = mt.f1_score(y_test, predictions_etd)

print("precision:",round(precison_etd,3),"recall:",round(recall_etd,3),"F1:",round(score_etd,3))

precision: 0.542 recall: 0.559 F1: 0.55


In [110]:
# PS results 

df_test_random = pd.read_csv("experiment/iterative/PS/ps_iter_result(t=0.8).csv")
predictions_ps = df_test_random["y''_PS"]
y_test = df_test_random["y_PS"]
precison_ps = mt.precision_score(y_test, predictions_ps)
recall_ps = mt.recall_score(y_test, predictions_ps)
score_ps = mt.f1_score(y_test, predictions_ps)

print("precision:",round(precison_ps,3),"recall:",round(recall_ps,3),"F1:",round(score_ps,3))

precision: 0.67 recall: 0.795 F1: 0.727
